In [1]:
import cx_Oracle
import os
import pandas as pd
#!pip insall cx_Oracle
import xlwings as xw
import pandas as pd
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

In [35]:
connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

## new_data_input_to_table

1. 새로운 데이터 입력시 edit_mode에서 선택 가능
2. data_input_mode 를 만들어 해당 모드에서 진행한다.
    - edit_mode 에서만 선택가능, data_insert_mode 로 진행한다.
    - data_insert() 매서드로 db저장 및 xl_table 구현을 한다.
2. db상의 index 마지막 값을 찾아 그 이후 값부터 부여한다.
    - 사용자가 임의로 해당 셀에 값을 주었다고 하더라도 지우고 정해진 index값으로 넣는다.
3. data_input_mode에서는 table의 모든 값을 지운다.

### 과정
- data_insert_mode()
1. edit_mode 에서만 진행가능
    - 다른 모드에서는 edit_mode에서만 진행 가능함을 알림
2. table 내용 모두 삭제
3. status는 data_insert_mode로 변경


- data_input()
1. data 입력시작 
2. data 입력이 완료 되면, db에서 가장 높은 index값을 찾아 그값보다 높은 값을 순서대로 부여하여 db에 저장
3. db저장이 완료되면, 다시 data_insert_mode 종료 및 edit_mode 종료 
4. table에 다시 데이터를 불러들여온다.


In [50]:
sel_sht = wb_cy.selection.sheet
last_col_num =  sel_sht.range("XFD9").end('left').column

In [51]:
col_names = sel_sht.range((9,1),(9,last_col_num)).value
col_names

['SI_INDEX',
 'AWB_NO',
 'TRIP_NO',
 'SHIPMENT_NM',
 'NM_OF_PACKAGE',
 'INVOICE_DATE',
 'ORDER_NM',
 'ORDER_TOTAL',
 'UNIT_PRICE',
 'SHIP_TO',
 'ARRIVAL_DATE',
 'SHIP_DATE',
 'POD_DATE',
 'FOR_FREE',
 'REMARK',
 'PARCELS_NO',
 'COMMENT',
 'STATUS']

In [68]:
si_db = cursor.execute('select * from shipment_information')

In [69]:
df_si = pd.DataFrame(si_db.fetchall(),columns=col_names)
df_si

,SI_INDEX,AWB_NO,TRIP_NO,SHIPMENT_NM,NM_OF_PACKAGE,INVOICE_DATE,ORDER_NM,ORDER_TOTAL,UNIT_PRICE,SHIP_TO,ARRIVAL_DATE,SHIP_DATE,POD_DATE,FOR_FREE,REMARK,PARCELS_NO,COMMENT,STATUS
0,69,1041685812,None,None,1,None,None,None,None,삼성바이오로직스,2021-11-01,2021-11-02,SVC,None,None,None,삼성배송예정,None
1,70,3408190085,None,None,1,None,None,None,None,DHL특송,2021-11-02,2021-11-03,None,None,None,None,None,None
2,71,5379068192,None,None,1,None,None,None,None,DHL특송,2021-11-02,2021-11-03,None,None,None,None,None,None
3,72,1807999130,None,None,1,None,None,None,None,FTC,2021-11-09,2021-11-09,None,None,None,None,수리 후 FTC 택배발송,None
4,73,951869855693,106199286,123436073,1,None,2119970,None,None,서린바이오사이언스,2021-11-10,2021-11-10,2021-11-12,None,대리점,PYD002672682,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20482,19904,STO0165331,115379865,139820494,1,2022-10-11,2121926,313.3,None,동일과학,2022-10-19,2022-10-24,None,None,대리점,UPD3345577,None,None
20483,19905,STO0165331,115379865,139820494,None,2022-10-11,2123091,313.3,None,동일과학,2022-10-19,2022-10-24,None,None,대리점,UPD3345577,None,None
20484,19906,STO0165331,115379865,139820494,None,2022-10-11,2123205,313.3,None,동일과학,2022-10-19,2022-10-24,None,None,대리점,UPD3345577,None,None
20485,19907,STO0165331,115379865,139820494,1,2022-10-11,2123806,626.6,None,동일과학,2022-10-19,2022-10-24,None,None,대리점,UPD3345690,None,None


In [70]:
df_si = df_si.sort_values('SI_INDEX')
df_si = df_si.replace('None','')

In [71]:
df_si = df_si.reset_index(drop=True)
df_si= df_si.set_index("SI_INDEX")


In [72]:
sel_sht.range("A9").value = df_si

In [9]:
sel_sth = wb_cy.selection.sheet
status_col_nm =sel_sth.range("XFD4").end("left").column

In [32]:
cur = cursor

In [33]:
pd.DataFrame(cur.execute('select from shipment_information').fetchall())

20487

In [37]:
def data_insert():
    sel_sth = wb_cy.selection.sheet
    status_col_nm =sel_sth.range("XFD4").end("left").column
    status = sel_sth.range(4,status_col_nm)
    if status.value == 'edit_mode':  # edit_mode에서만 data_input_mode() 사용 가능
        last_row = sel_sth.range("A1048576").end('up').row
        last_col = sel_sth.range("XFD9").end("left").column
        xl_table = sel_sth.range((10,1),(last_row,last_col))
        xl_table.clear_contents()
        status.value = 'data_insert_mode'
        wb_cy.app.alert("data_insert_mode모드로 진입합니다. 데이터 입력 후 버튼을 다시한번 눌러주세요.","DATA Input Mode")
        return None
    elif status.value == 'data_insert_mode':  # data_insert_mode 상태면 다시 data_input기능을 마칠지 물어본다.
        input_comfirm = wb_cy.app.alert("'AWB'컬럼은 빈칸이 없어야 정상작동 합니다. 입력한 DATA를 Confirm하시겠습니까?","Input Confirm")
        if input_comfirm == True: # data input 시작
            # 입력한 값이 있는지는 확인해봐야함
            last_row = sel_sth.range("B1048576").end('up').row
            last_col = sel_sth.range("XFD9").end("left").column
            if last_row > 9: # 10번 행에 값이 있다는 뜻.. data_input실행가능
                data_input() # data db업로드 완료
                # db에서 해당 테이블 모든 데이터 불러와서 xl_si_sht로 데이터 전송
                df_si = pd.DataFrame(cur.execute('select * from shipment_information').fetchall())
                col_names = sel_sth.range((9,1),(9,last_col)).value
                df_si.columns=col_names
                df_si.set_index('SI_INDEX',inplace=True)
                df_si = df_si.replace('None','')
                sel_sth.range("A9").value =df_si
                
                # edit_mode로 돌아가기
                
            else :
                wb_cy.app.alert("Input할 값이 없습니다. 'AWB'컬럼에는 반드시 값이 있어야합니다..","Input WARNING")
                return None


        else : #False #취소이므로 매서드 중단
            wb_cy.app.alert("DATA Input을 중단합니다.","Input Confirm")
            return None
        
    else : 
        wb_cy.app.alert("해당 기능은 'edit_mode'상태에서만 사용가능합니다.","WARNING")
        return None







True


In [107]:
# data임의 삽입
df_si = pd.DataFrame(cur.execute('select * from shipment_information').fetchall())
col_names = sel_sth.range((9,1),(9,last_col)).value
df_si.columns=col_names
df_si = df_si.sort_values('SI_INDEX')
df_si.set_index('SI_INDEX',inplace=True)
df_si = df_si.replace('None','')
df_si
sel_sth.range("A9").value = df_si

In [43]:
## data_input 컨펌모드
# data를 확인하고 가져와야하는데 AWB cell은 NOT NULL이다

In [54]:
sel_sth = wb_cy.selection.sheet
input_comfirm = wb_cy.app.alert("'AWB'컬럼은 빈칸이 없어야 정상작동 합니다. 입력한 DATA를 Confirm하시겠습니까?","Input Confirm")
if input_comfirm == True: # data input 시작
    # 입력한 값이 있는지는 확인해봐야함
    last_row = sel_sth.range("B1048576").end('up').row
    last_col = sel_sth.range("XFD9").end("left").column
    if last_row > 9: # 10번 행에 값이 있다는 뜻.. data_input실행가능
        col_names = sel_sth.range((9,1),(9,last_col)).value
        xl_content = sel_sth.range((10,1),(last_row,last_col)).value
        df_input = pd.DataFrame(xl_content,columns=col_names)
    
    
else : #False #취소이므로 매서드 중단
    wb_cy.app.alert("DATA Input을 중단합니다.","Input Confirm")

20487

In [15]:
import datetime as dt
sel_sth = wb_cy.selection.sheet

In [23]:
1.0 == int(1)

True

In [34]:
last_row = sel_sth.range("A1048576").end('up').row
last_col = sel_sth.range("XFD9").end("left").column
col_names_1 = sel_sth.range((9,1),(9,7)).options(numbers=int,dates=dt.date).value
content_1 = sel_sth.range((10,1),(last_row,7)).options(numbers=int,dates=dt.date).value
col_names_2 = sel_sth.range((9,8),(9,9)).options(numbers=int,dates=dt.date).value
content_2 = sel_sth.range((10,8),(last_row,9)).options(dates=dt.date).value
col_names_3 = sel_sth.range((9,10),(9,last_col)).options(numbers=int,dates=dt.date).value
content_3 = sel_sth.range((10,10),(last_row,last_col)).options(numbers=int,dates=dt.date).value
df_1 = pd.DataFrame(content_1,columns=col_names_1)
df_2 = pd.DataFrame(content_2,columns=col_names_2)
df_3 = pd.DataFrame(content_3,columns=col_names_3)
df = pd.concat([df_1,df_2,df_3],axis=1)

df = df.astype('string')
df = df.astype({
    'SI_INDEX':'int'
})
df = df.fillna('')
df
# df['SI_INDEX'] = None
# df_len = len(df)
# df['SI_INDEX'] = [*range(last_idx_db+1,last_idx_db+df_len+1)]
# df



,SI_INDEX,AWB_NO,TRIP_NO,SHIPMENT_NM,NM_OF_PACKAGE,INVOICE_DATE,ORDER_NM,ORDER_TOTAL,UNIT_PRICE,SHIP_TO,ARRIVAL_DATE,SHIP_DATE,POD_DATE,FOR_FREE,REMARK,PARCELS_NO,COMMENT,STATE,TIMELINE
0,1,7764059026,,,,,,,,DHL특송건,2021-01-04,2021-01-06,,무상,,,,,
1,2,9531213646,,,,,,,,DHL특송건,2021-01-04,2021-01-08,,,,,,,
2,3,437779237520,105354422,115291677,1.0,,2116872,205.65,205.65,바이나리,2021-01-18,2021-01-18,2021-01-20,,대리점,PYD002357006,,,
3,4,STO0148367,105359564,115374865,,,2116946,1765.74,1765.74,이효바이오사이언스,2021-01-19,2021-01-19,2021-01-20,,대리점,UPD3047151,,,
4,5,STO0148367,105359564,115374865,,,2116947,165.98,165.98,이효바이오사이언스,2021-01-19,2021-01-19,2021-01-20,,대리점,UPD3047151,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20482,20483,STO0165597,115394010,140138477,1.0,2022-10-24,2125298,2017.48,2017.48,바이나리,2022-10-31,,,,대리점,UPD3351722,,,
20483,20484,STO0165597,115394011,140137540,1.0,2022-10-24,800665,8726.58,,비엔씨코리아,2022-10-31,,,,SHL_Microzyme_Q-298606 V0_AKTApure150M,UPD3351766,,,
20484,20485,STO0165597,115394011,140149337,1.0,2022-10-24,800665,,,비엔씨코리아,2022-10-31,,,,SHL_Microzyme_Q-298606 V0_AKTApure150M,UPD3352123,,,
20485,20486,STO0165597,115394012,140139530,1.0,2022-10-24,IR-SM00515698,2699.07,,KR_CCK,2022-10-31,,,,,UPD3351878,,,


0        1
1        1
2        1
3        1
4        1
        ..
20482    1
20483    1
20484    1
20485    1
20486    1
Name: TIMELINE, Length: 20487, dtype: int64

In [142]:
df_1 = pd.DataFrame([content_1],columns=col_names_1)


,SI_INDEX,AWB_NO,TRIP_NO,SHIPMENT_NM,NM_OF_PACKAGE,INVOICE_DATE,ORDER_NM
0,1,1,1,1,1,1899-12-31,1


In [103]:
def data_input():
    last_idx_db = pd.DataFrame(cur.execute('select max(si_index) from shipment_information').fetchall())[0][0]
    last_row = sel_sth.range("B1048576").end('up').row
    last_col = sel_sth.range("XFD9").end("left").column
    col_names = sel_sth.range((9,1),(9,last_col)).value
    xl_content = sel_sth.range((10,1),(last_row,last_col)).value
    col_names_1 = sel_sth.range((9,1),(9,7)).options(numbers=int).value
    content_1 = sel_sth.range((10,1),(last_row,7)).options(numbers=int).value
    col_names_2 = sel_sth.range((9,8),(9,9)).value
    content_2 = sel_sth.range((10,8),(last_row,9)).value
    col_names_3 = sel_sth.range((9,10),(9,last_col)).options(numbers=int,dates=dt.date).value
    content_3 = sel_sth.range((10,10),(last_row,last_col)).options(numbers=int,dates=dt.date).value
    df_1 = pd.DataFrame(content_1,columns=col_names_1)
    df_2 = pd.DataFrame(content_2,columns=col_names_2)
    df_3 = pd.DataFrame(content_3,columns=col_names_3)
    df = pd.concat([df_1,df_2,df_3],axis=1)
    df = df.astype('string')
    df = df.astype({
        'SI_INDEX':'int'
    })
    df = df.fillna('None')
    df['SI_INDEX'] = None
    df_len = len(df)
    df['SI_INDEX'] = [*range(last_idx_db+1,last_idx_db+df_len+1)]

,SI_INDEX,AWB_NO,TRIP_NO,SHIPMENT_NM,NM_OF_PACKAGE,INVOICE_DATE,ORDER_NM,ORDER_TOTAL,UNIT_PRICE,SHIP_TO,ARRIVAL_DATE,SHIP_DATE,POD_DATE,FOR_FREE,REMARK,PARCELS_NO,COMMENT,STATUS
0,20488,STO0165246,115368327,139715606,1,2022-10-06,2124747,4353.28,4081.2,서린바이오사이언스,2022-10-14,2022-10-17,None,None,대리점,UPD3343683,None,None
1,20489,STO0165246,115368327,139715606,1,2022-10-06,2124747,None,272.08,서린바이오사이언스,2022-10-14,2022-10-17,None,None,대리점,UPD3343607,None,None
2,20490,STO0165246,115368319,139707952,1,2022-10-06,2119572,15917.35,None,진메디신,2022-10-14,2022-10-26,None,None,SHL_Genemedi_CaptoQimpres,UPD3343396,None,None
3,20491,STO0165246,115368320,139706899,1,2022-10-06,2125047,893.99,None,SK바이오사이언스,2022-10-14,2022-10-21,None,None,7700186166,UPD3343380,None,None
4,20492,STO0165246,115368321,139710190,1,2022-10-06,800586,51160.94,None,비엠아이,2022-10-14,None,None,None,MGK_BMI_Q-264311 V0_AKTA,UPD3343531,None,None


In [92]:
last_idx_db

20487

In [93]:
df_len = len(df)
range(last_idx_db+1,last_idx_db+df_len+1)

range(20487, 5)

In [99]:
for i in range(last_idx_db+1,last_idx_db+df_len+1):
    print(i)

20488
20489
20490
20491
20492


In [112]:
wb_cy.app.alert("맞아?",buttons="yes_no_cancel") 

'cancel'

In [ ]:
# data_input 끝나고 전체 데이터 출력 시 폰트 및 서식 맞추기


In [159]:
last_row = sel_sth.range("B1048576").end('up').row
last_col = sel_sth.range("XFD9").end("left").column
col_names = sel_sth.range((9,1),(9,last_col)).options(numbers=int,dates=dt.date).value
xl_content = sel_sth.range((10,1),(last_row,last_col)).options(numbers=int,dates=dt.date)
# df_si = pd.DataFrame(xl_content,columns=col_names)

In [147]:
xl_content_1=  sel_sth.range((10,1),(10,last_col)).options(numbers=int,dates=dt.date)

In [179]:
# 테두리 검은색
xl_content.api.Borders.LineStyle = 1

In [206]:
xl_content.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignLeft


In [216]:
# 데이터 진입모드시 입력할 첫째 행으로 포인터 이동 필수..
# index컬럼에 값이 없다면 매서드 진행 불가로 index에는 자동으로 값을 넣어주자 최대 1000개
tmp_idx = [*range(1,1000)]

sel_sth.range("A10").select()

In [219]:
sel_sth.range("A9").value = tmp_col

In [217]:
sel_sth.range("A10").options(transpose=True).value = tmp_idx

In [218]:
tmp_col =['SI_INDEX'
,'AWB_NO'
,'TRIP_NO'
,'SHIPMENT_NM'
,'NM_OF_PACKAGE'
,'INVOICE_DATE'
,'ORDER_NM'
,'ORDER_TOTAL'
,'UNIT_PRICE'
,'SHIP_TO'
,'ARRIVAL_DATE'
,'SHIP_DATE'
,'POD_DATE'
,'FOR_FREE'
,'REMARK'
,'PARCELS_NO'
,'COMMENT'
,'STATUS']